In [ ]:
# (planta, [personas], [ascensores])
# ascensores_lentos = 0,1,2,3  ascensor_rapido = 4
estado_ini = {
    (0, [], [0,4])
    (1, [2,4], []),
    (2, [0], []),
    (3, [], []),
    (4, [1], [1,2]),
    (5, [], []),
    (6, [], []),
    (7, [], []),
    (8, [3], [3]),
    (9, [], []),
    (10, [], []),
    (11, [], []),
    (12, [], [])
}

estado_fin = {}
    (0, [], [])
    (1, [3], []),
    (2, [], []),
    (3, [0], []),
    (4, [], []),
    (5, [], []),
    (6, [], []),
    (7, [], []),
    (8, [], []),
    (9, [4], []),
    (10, [], []),
    (11, [1], []),
    (12, [2], [])
}

'''self._actions = [('a',(0,0)), ('a',(0,1)), ('a',(0,2)), ('a',(0,3)), ('a',(0,4)),
                         ('a',(1,4)), ('a',(1,5)), ('a',(1,6)), ('a',(1,7)), ('a',(1,8)),
                         ('a',(2,4)), ('a',(2,5)), ('a',(2,6)), ('a',(2,7)), ('a',(2,8)),
                         ('a',(3,8)), ('a',(3,9)), ('a',(3,10)), ('a',(3,11)), ('a',(3,12)),
                         ('a',(4,0)), ('a',(4,2)), ('a',(4,4)), ('a',(4,6)), ('a',(4,8)), ('a',(4,10)), ('a',(4,12)),
                         ('p',(0,0)), ('p',(0,1)), ('p',(0,2)), ('p',(0,3)), ('p',(0,4)),
                         ('p',(1,0)), ('p',(1,1)), ('p',(1,2)), ('p',(1,3)), ('p',(1,4)),
                         ('p',(2,0)), ('p',(2,1)), ('p',(2,2)), ('p',(2,3)), ('p',(2,4)),
                         ('p',(3,0)), ('p',(3,1)), ('p',(3,2)), ('p',(3,3)), ('p',(3,4)),
                         ('p',(4,0)), ('p',(4,1)), ('p',(4,2)), ('p',(4,3)), ('p',(4,4)),
                        ]'''
'''self._actions = [('a',(0,0)), ('a',(0,1)), ('a',(0,2)), ('a',(0,3)), ('a',(0,4)),
                         ('a',(1,4)), ('a',(1,5)), ('a',(1,6)), ('a',(1,7)), ('a',(1,8)),
                         ('a',(3,8)), ('a',(3,9)), ('a',(3,10)), ('a',(3,11)), ('a',(3,12)),
                         ('a',(4,0)), ('a',(4,2)), ('a',(4,4)), ('a',(4,6)), ('a',(4,8)), ('a',(4,10)), ('a',(4,12)),
                         ('p',(0,0)), ('p',(0,1)), ('p',(0,2)), ('p',(0,3)), ('p',(0,4)),
                         ('p',(1,0)), ('p',(1,1)), ('p',(1,2)), ('p',(1,3)), ('p',(1,4)),
                         ('p',(2,0)), ('p',(2,1)), ('p',(2,2)), ('p',(2,3)), ('p',(2,4)),
                        ]'''

In [6]:
# 1 persona, 1 ascensor
# estado_ini
personas_ini = (2)
ascensores_ini = (0)

# estado_fin
personas_goal = (10)
ascensores_goal = ()

# Acciones
# Llamar ascensor (mueve un ascensor vacio a una planta)
# Subir persona (sube una persona dentro de un ascensor a una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar persona (baja una persona dentro de un ascensor a una planta) (comprobar si ascensor y persona estan en misma planta)

In [170]:
# importamos las cosas que vamos a usar de aima
from search import *

class Ascensores(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal):
        '''Inicializacion de nuestro problema.'''
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        p = list(estado[0])
        a = list(estado[1])
        accs = list()
        if p[0] != a[0]:
            accs.append(("Llamar ",str(p[0])));
        if p[0] == a[0]:
            accs.append(("Bajar", str(a[0]-1)))
            accs.append(("Subir", str(a[0]+1)))
        return accs
    
    def result(self, estado, accion):
        p = list(estado[0])
        a = list(estado[1])
        if accion[0] == "Llamar":
            a[0] = accion[1]
        elif accion[0] == "Bajar":
            a[0] = accion[1]
            p[0] = accion[1]
        elif accion[0] == "Subir":
            a[0] = accion[1]
            p[0] = accion[1]
        return (p,a)


In [182]:
p = Ascensores(([2],[0]),(10))
p.actions(p.initial)

[('Llamar ', '2')]

In [183]:
p.result(p.initial, ("Llamar",1))

([2], [1])

In [173]:
p.goal

10

In [174]:
p.initial

([2], [2])

In [175]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [176]:
p = Ascensores(([2],[2]),(10))
breadth_first_tree_search(p).solution()

TypeError: unsupported operand type(s) for -: 'str' and 'int'